<a href="https://colab.research.google.com/github/zetavg/LLM-Research/blob/main/TW_Pythia_Embedding_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import transformers, accelerate, bitsandbytes
except:
    !pip install transformers accelerate bitsandbytes


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [2]:
import gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

low_gpu_ram_usage = True
if low_gpu_ram_usage:
    if 'model_without_zhtw_tokens_training' in globals():
        del model_without_zhtw_tokens_training
    gc.collect()
    with torch.no_grad():
        torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained('twlm/tw-pythia-6.9b-chat-v0_2')
model = AutoModelForCausalLM.from_pretrained(
    'twlm/tw-pythia-6.9b-chat-v0_2',
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map={'': 'cuda'},
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
if 'model' not in globals():
    model = None

def get_embedding(word, tokenizer=tokenizer, model=model):
    ids = tokenizer.encode(word, return_tensors="pt")[0].to('cuda')
    if ids.shape[0] > 1:
        raise ValueError(f"The word '{word}' is consisted with more than 1 token: {ids}.")
    input_embeddings = model.get_input_embeddings()(ids)
    return input_embeddings[0]

# Sample
get_embedding(' cat')

In [4]:
import torch
from torch.nn.functional import cosine_similarity

def get_cosine_similarity(tensor_1, tensor_2):
    similarity = cosine_similarity(tensor_1.unsqueeze(0), tensor_2.unsqueeze(0))
    return similarity.item()


# Sample
a = torch.tensor([   1,    0, -1   ], dtype=torch.float16).to('cuda')
b = torch.tensor([ 1.1,    0, -1   ], dtype=torch.float16).to('cuda')
c = torch.tensor([ 0.1, -0.1, -0.8 ], dtype=torch.float16).to('cuda')
d = torch.tensor([  -1,    0, 1    ], dtype=torch.float16).to('cuda')
print('Cosine similarity of a and b: ', get_cosine_similarity(a, b))
print('Cosine similarity of a and c: ', get_cosine_similarity(a, c))
print('Cosine similarity of a and d: ', get_cosine_similarity(a, d))

Cosine similarity of a and b:  0.9990234375
Cosine similarity of a and c:  0.783203125
Cosine similarity of a and d:  -1.0


In [5]:
get_cosine_similarity(
    get_embedding(' cat'),
    get_embedding(' dog'),
)

0.1405029296875

In [6]:
get_cosine_similarity(
    get_embedding(' cat'),
    get_embedding(' car'),
)

0.053192138671875

In [7]:
get_cosine_similarity(
    get_embedding(' dog'),
    get_embedding(' god'),
)

0.0792236328125

In [8]:
get_cosine_similarity(
    get_embedding(' cat'),
    get_embedding('貓'),
)

0.0888671875

In [9]:
get_cosine_similarity(
    get_embedding(' dog'),
    get_embedding('狗'),
)

0.099853515625

In [10]:
get_cosine_similarity(
    get_embedding(' walk'),
    get_embedding('走'),
)

0.0711669921875

In [11]:
get_cosine_similarity(
    get_embedding('貓'),
    get_embedding('狗'),
)

0.0838623046875

In [12]:
get_cosine_similarity(
    get_embedding('狗'),
    get_embedding('皇'),
)

0.006008148193359375

In [13]:
get_cosine_similarity(
    get_embedding('貓'),
    get_embedding('皇'),
)

0.0190277099609375

In [14]:
get_cosine_similarity(
    get_embedding('狗'),
    get_embedding('神'),
)

-0.003875732421875

In [15]:
get_cosine_similarity(
    get_embedding('貓'),
    get_embedding('喵'),
)

0.0156097412109375

In [16]:
get_cosine_similarity(
    get_embedding('狗'),
    get_embedding('喵'),
)

-0.01073455810546875

In [17]:
get_cosine_similarity(
    get_embedding('狗'),
    get_embedding('汪'),
)

0.041290283203125

In [1]:
import gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

low_gpu_ram_usage = True
if low_gpu_ram_usage:
    if 'model' in globals():
        del model
    if 'model_without_zhtw_tokens_training' in globals():
        del model_without_zhtw_tokens_training
    gc.collect()
    with torch.no_grad():
        torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained('twlm/tw-pythia-6.9b-chat-v0_2')
model_without_zhtw_tokens_training = AutoModelForCausalLM.from_pretrained(
    'zetavg/pythia-6.9b',
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map={'': 'cuda'},
)
model_without_zhtw_tokens_training.resize_token_embeddings(tokenizer.vocab_size)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Embedding(58113, 4096)

In [5]:
get_cosine_similarity(
    get_embedding(' cat', model=model_without_zhtw_tokens_training),
    get_embedding('貓', model=model_without_zhtw_tokens_training),
)

-0.01605224609375

In [6]:
get_cosine_similarity(
    get_embedding(' dog', model=model_without_zhtw_tokens_training),
    get_embedding('狗', model=model_without_zhtw_tokens_training),
)

0.0163116455078125

In [7]:
get_cosine_similarity(
    get_embedding('貓', model=model_without_zhtw_tokens_training),
    get_embedding('狗', model=model_without_zhtw_tokens_training),
)

-0.009368896484375

In [8]:
get_cosine_similarity(
    get_embedding('冷', model=model_without_zhtw_tokens_training),
    get_embedding('狗', model=model_without_zhtw_tokens_training),
)

-0.005687713623046875